# Setup

In [9]:
import sys
assert sys.version_info >= (3, 8)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from utils import display_images, matplotlib_show_images, display_images, display_image, auto_canny
if cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Weekly activities
1. Apply custom sharpening kernel of aperture size 3 and 5 as shown below on 'native-bee.png':  
$ 3 \times 3$ kernel:  
$ \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0\\
\end{bmatrix}$  
$ 5 \times 5$ kernel:  
$ \begin{bmatrix}
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & 25 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
\end{bmatrix}$  
What can you infer from the outputs?
2. Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.
3. Write a program to *segment the boat and the people on it from the background*. Follow the instruction below:
    - Use 'boat.jpg' as input.
    - Apply Otsu thresholding.
    - Draw bounding box to identify the region where the boat and people are located.

In [3]:
#1 
kernel1 = np.array ([[0, -1, 0],
                   [-1, 5, -1],
                   [0, -1, 0]])

kernel2 = np.array([[-1, -1, -1, -1, -1],
                   [-1, -1, -1, -1, -1],
                   [-1, -1, 25, -1, -1],
                   [-1, -1, -1, -1, -1],
                   [-1, -1, -1, -1, -1]])

img = cv.imread("images/native-bee.png")
dst1 = cv.filter2D(img, -1, kernel1)
dst2 = cv.filter2D(img, -1, kernel2)

display_images([img, dst1, dst2], ("source","3x3", "5x5"))

# moderate sharpening effect for 3x3 kernel
# strong sharpening for 5x5 kernel, but increase noise

In [7]:
#2
img = cv.imread("images/noise_lena.jpg")

#average filter
blur1 = cv.blur(img, (5, 5))

#Gaussian kernel
blur2 = cv.GaussianBlur(img, (5, 5), 0)

#median filter
blur3 = cv.medianBlur(img, 5)

#bilateral filtering
blur4 = cv.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

display_images([img, blur1, blur2, blur3, blur4],("source", "average_filter", "Gaussian kernel", "median filter", "bilateral filtering"))

# average filter = noise reduced but blur edge, less sharp
# Gaussian kernel = reduce noise more natural but a little bit of edge blurring, not suit for salt-and-pepper noise
# Median filter = preserves edge better than average and Gaussian filter, most suitable for salt-and-pepper noise
# Bilateral filter = removed noise while preserving edges and sharpness, still some noise present compare to median filter

# median >> bilateral > gaussian >≈ average

# The type of noise on noise_lena.jpg is salt-and-pepper as characterized by a random apperance of black and white pixels throughout the image

In [43]:
#3
img = cv.imread("images/boat.jpg")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# applying otsu thresholding
ret, th = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

# contour
contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#post-processing
contour_max_area = max(contours, key=cv.contourArea)

#draw bounding box
x, y, w, h = cv.boundingRect(contour_max_area)
img_copy = img.copy()
# cv.drawContours(img_copy, [contour_max_area], -1, (0, 255, 0), 2)
cv.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
display_image("gotchu boat", img_copy)

In [23]:
img.shape

(514, 768, 3)

# Useful links:
- Learn more about different types of image filters: https://setosa.io/ev/image-kernels/